In [7]:
from osgeo import gdal, ogr, osr
from osgeo.gdalconst import *
import numpy as np
import sys
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
gdal.PushErrorHandler('CPLQuietErrorHandler')
%matplotlib inline

In [8]:
def bbox_to_pixel_offsets(gt, bbox):
    originX = gt[0]
    originY = gt[3]
    pixel_width = gt[1]
    pixel_height = gt[5]
    x1 = int((bbox[0] - originX) / pixel_width)
    x2 = int((bbox[1] - originX) / pixel_width) + 1

    y1 = int((bbox[3] - originY) / pixel_height)
    y2 = int((bbox[2] - originY) / pixel_height) + 1

    xsize = x2 - x1
    ysize = y2 - y1
    return (x1, y1, xsize, ysize)

In [94]:
def zonal_stats(vector_path, raster_path, nodata_value):
    
    # open raster layer
    rds = gdal.Open(raster_path, GA_ReadOnly)
    assert(rds)
    rb = rds.GetRasterBand(1)
    rgt = rds.GetGeoTransform()
    
    # set raster nodata value
    if nodata_value:
        nodata_value = float(nodata_value)
        rb.SetNoDataValue(nodata_value)
    
    # open vector layer
    vds = ogr.Open(vector_path, GA_ReadOnly)  
    assert(vds)
    vlyr = vds.GetLayer(0)    
    
    # compare EPSG values of vector and raster and change projection if necessary
    sourceSR = vlyr.GetSpatialRef()
    sourceSR.AutoIdentifyEPSG()
    EPSG_sourceSR = sourceSR.GetAuthorityCode(None)
    
    targetSR = osr.SpatialReference(wkt=rds.GetProjection())
    targetSR.AutoIdentifyEPSG()
    EPSG_targetSR = targetSR.GetAuthorityCode(None)
    
    if EPSG_sourceSR != EPSG_sourceSR:
        # reproject vector geometry to same projection as raster
        print 'unequal projections'    
        sourceSR = vlyr.GetSpatialRef()
        targetSR = osr.SpatialReference()
        targetSR.ImportFromWkt(rds.GetProjectionRef())
        coordTrans = osr.CreateCoordinateTransformation(sourceSR,targetSR)    
        
    """do the work"""
    global_src_extent = None
    mem_drv = ogr.GetDriverByName('Memory')
    driver = gdal.GetDriverByName('MEM')
    
    # Loop through vectors
    stats = []
    feat = vlyr.GetNextFeature() 
    
    while feat is not None:
        # print statement after each hunderds features
        fid = int(feat.GetFID())  
        prov = str(feat.GetField('NAME_1'))
        print("start with feature %s " % (fid)) 
        print("province: %s " % (prov)) 
        #if fid % 500 == 0:
        #    print("finished first %s features" % (fid))
    
        if not global_src_extent:
            #print 'bbox county'
            # use local source extent
            # fastest option when you have fast disks and well indexed raster (ie tiled Geotiff)
            # advantage: each feature uses the smallest raster chunk
            # disadvantage: lots of reads on the source raster
            src_offset = bbox_to_pixel_offsets(rgt, feat.geometry().GetEnvelope())
            src_array = rb.ReadAsArray(*src_offset)
        
            # calculate new geotransform of the feature subset
            new_gt = (
                (rgt[0] + (src_offset[0] * rgt[1])),
                rgt[1],
                0.0,
                (rgt[3] + (src_offset[1] * rgt[5])),
                0.0,
                rgt[5]
            )
        
        # Create a temporary vector layer in memory
        mem_ds = mem_drv.CreateDataSource('out')
        mem_layer = mem_ds.CreateLayer('poly', None, ogr.wkbPolygon)
        mem_layer.CreateFeature(feat.Clone())
        
        # Rasterize it
        rvds = driver.Create('', src_offset[2], src_offset[3], 1, gdal.GDT_Byte)
        rvds.SetGeoTransform(new_gt)
        gdal.RasterizeLayer(rvds, [1], mem_layer, burn_values=[1])
        rv_array = rvds.ReadAsArray()
        
        # Mask the source data array with our current feature
        # we take the logical_not to flip 0<->1 to get the correct mask effect
        # we also mask out nodata values explictly
        try:
            masked = np.ma.MaskedArray(
                src_array,
                mask = np.logical_or(
                    src_array == nodata_value,
                    np.logical_not(rv_array)
                )
            )
            
            #print 'feature ID: ',int(feat.GetFID())
            
            # GET STATISTICS FOR EACH COUNTY
            print 'do something'
            print masked.shape
            #return masked
            values, counts = np.unique(masked, return_counts=True)
            print values, counts
            pixels = counts[0]
            total_pixels_prov = np.ma.count(masked)
            print pixels
            comb = [prov,pixels,total_pixels_prov]
            
            #county_stats = getStatsCounty(cnty_array = masked, feat=feat)            
            stats.append(comb)
            
            rvds = None
            mem_ds = None
            feat = vlyr.GetNextFeature()
            
        except np.ma.MaskError: 
            # catch MaskError, ignore feature containing no valid corresponding raster data set
            # in my case the the most southern county of hainan is not totally within the raster extent            
            print 'feature ID: ',fid, ' maskError, ignore county and lets continue'
            
            rvds = None
            mem_ds = None
            feat = vlyr.GetNextFeature()            
    
    vds = None
    rds = None
    return stats#, src_array, rv_array, masked

In [99]:
vector_path = r'D:\Data\ChinaShapefile\CHN_adm//CHN_adm1.shp'
raster_path = r'D:\Data\ChinaWorld_GlobCover//CN_VEGETATION_CROP.tif'
nodata_value = np.nan

In [100]:
stats = zonal_stats(vector_path, raster_path, nodata_value)

start with feature 0 
province: Anhui 
do something
(1895L, 1718L)
[1 -- 127 ..., -- 127 --] [1644578      13       1 ...,     167       4      96]
1644578
start with feature 1 
province: Beijing 
do something
(583L, 752L)
[1 -- 127 ..., -- 127 --] [199069      3      2 ...,      1     36    149]
199069
start with feature 2 
province: Chongqing 
do something
(1458L, 1767L)
[1 127 -- ..., -- 127 --] [987722      7      1 ...,      1      1     30]
987722
start with feature 3 
province: Fujian 
do something
(1723L, 1798L)
[1 -- 127 ..., -- 127 --] [1391244      81       1 ...,      31       2      86]
1391244
start with feature 4 
province: Gansu 
do something
(3666L, 5744L)
[1 -- 127 ..., -- 127 --] [2629513      18       1 ...,       5       2    2216]
2629513
start with feature 5 
province: Guangdong 
do something
(1908L, 2754L)
[1 -- 127 ..., -- 127 --] [1914036      11       1 ...,       1      13     550]
1914036
start with feature 6 
province: Guangxi 
do something
(1976L, 2728L)


In [101]:
df = pd.DataFrame(stats)
df.columns=['prov','veg','total']
df.set_index(['prov'], inplace=True)
df

,veg,total
prov,,
Anhui,1644578,1732556
Beijing,199069,224841
Chongqing,987722,999621
Fujian,1391244,1430279
Gansu,2629513,5378923
Guangdong,1914036,2035242
Guangxi,2674713,2716493
Guizhou,2062819,2070776
Hainan,366113,379192


In [98]:
df2['crop'].to_csv(r'D:\Data\NDAI_VHI_GROUNDTRUTH//cropland.csv')

In [105]:
df3 = pd.concat([df['veg'],df2],axis=1)

In [112]:
df3

,veg,crop,total
Anhui,1644578,1088107,1732588
Beijing,199069,85648,224853
Chongqing,987722,419977,999645
Fujian,1391244,100248,1430155
Gansu,2629513,1073330,5378893
Guangdong,1914036,221849,2035136
Guangxi,2674713,428743,2716511
Guizhou,2062819,770230,2070816
Hainan,366113,NaN,NaN
Hebei,2446417,1600189,2548354


In [111]:
(df3/30000).to_csv(r'D:\Data\NDAI_VHI_GROUNDTRUTH//crop_veg.csv')

In [104]:
df3['veg']-df3['crop']

Anhui              556471
Beijing            113421
Chongqing          567745
Fujian            1290996
Gansu             1556183
Guangdong         1692187
Guangxi           2245970
Guizhou           1292589
Hainan                NaN
Hebei              846228
Heilongjiang          NaN
Henan              424390
Hubei             1040828
Hunan             1470626
Jiangsu            128543
Jiangxi           1358920
Jilin             1828070
Liaoning          1022629
Nei Mongol        6993827
Ningxia Hui        241231
Qinghai           4624064
Shaanxi           1342683
Shandong           299802
Shanghai             6263
Shanxi             805321
Sichuan           4099817
Tianjin             44965
Xinjiang Uygur    3005923
Xizang            7507845
Yunnan            3442994
Zhejiang           802761
dtype: float64

In [27]:
values, counts = np.unique(masked, return_counts=True)
pixels = counts[0]
total_pixels_prov = np.ma.count(masked)

1732556

In [38]:
stats.size

3255610